# unit function test

In [1]:
from helpers.dataset import BldgDataset

dataset = BldgDataset("./DemoExperiment/1117/demo_test.zip")

Loading demo: 100%|██████████| 300/300 [00:01<00:00, 247.04path/s]


In [9]:
import numbers
import random


class RandomCropWithProb:
    def __init__(self, size, p=0.8, consistent=True):
        if isinstance(size, numbers.Number):
            self.size = (int(size), int(size))
        else:
            self.size = (size[0], size[1])
        self.consistent = consistent
        self.threshold = p

    def __call__(self, imgmap):
        img1 = imgmap[0]
        h, w = img1.shape
        print(w)
        if self.size is not None:
            th, tw = self.size
            print(tw)
            print(w - tw)
            if w == tw and h == th:
                return imgmap
            if self.consistent:
                if random.random() < self.threshold:
                    x1 = random.randint(0, w - tw)
                    y1 = random.randint(0, h - th)
                else:
                    x1 = int(round((w - tw) / 2.0))
                    y1 = int(round((h - th) / 2.0))
                return [i[y1 : y1 + th, x1 : x1 + tw] for i in imgmap]
            else:
                result = []
                for i in imgmap:
                    if random.random() < self.threshold:
                        x1 = random.randint(0, w - tw)
                        y1 = random.randint(0, h - th)
                    else:
                        x1 = int(round((w - tw) / 2.0))
                        y1 = int(round((h - th) / 2.0))
                    result.append(i[y1 : y1 + th, x1 : x1 + tw])
                return result
        else:
            return imgmap


In [18]:
import cv2
import numpy as np
import collections


class Scale:
    def __init__(self, size, interpolation=cv2.INTER_NEAREST):
        assert isinstance(size, int) or (
            isinstance(size, collections.Iterable) and len(size) == 2
        )
        self.size = size
        self.interpolation = interpolation

    def __call__(self, imgmap):
        resized_images = []
        for img in imgmap:
            h, w = img.shape[:2]
            if isinstance(self.size, int):
                if (w <= h and w == self.size) or (h <= w and h == self.size):
                    resized_images.append(img)
                    continue
                if w < h:
                    ow = self.size
                    oh = int(self.size * h / w)
                else:
                    oh = self.size
                    ow = int(self.size * w / h)
                resized_img = cv2.resize(
                    img, (ow, oh), interpolation=self.interpolation
                )
            else:
                resized_img = cv2.resize(
                    img, (self.size[1], self.size[0]), interpolation=self.interpolation
                )
            resized_images.append(resized_img)
        return resized_images


In [14]:
dataset = BldgDataset(data_path="./DemoExperiment/1117/demo_test.zip")

Loading demo: 100%|██████████| 300/300 [00:01<00:00, 261.44path/s]


In [18]:
import matplotlib.pyplot as plt

%matplotlib inline
test_data = dataset[0]["imgs"][:10]
print(test_data.shape)

cropped_transform = RandomCropWithProb(size=[25, 50], p=1.0, consistent=True)
scale_transform = Scale(size=(30, 60))
cropped_data = cropped_transform(test_data)
resized_data = scale_transform(cropped_data)
for i in range(len(resized_data)):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    img = cropped_data[i]
    ax.set_title(f"img {i}")
    ax.imshow(img, cmap="gray", vmin=0, vmax=255)
    plt.show()


(10, 30, 60)


ValueError: not enough values to unpack (expected 3, got 2)

In [21]:
import os
import shutil
import zipfile


def preprocess_image_data(base_path, zip_name):
    # Unzip the file
    zip_path = os.path.join(base_path, zip_name)
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(base_path)

    # Iterate through each 'pathXX' folder and rename PNGs using a temporary name
    for i in range(300):
        folder_name = f"path{str(i).zfill(2)}"
        folder_path = os.path.join(base_path, folder_name)

        # First, rename to temporary names
        for j in range(30):
            old_name = f"panoramic_{str(j).zfill(2)}.png"
            temp_name = f"temp_{str(j).zfill(2)}.png"
            old_path = os.path.join(folder_path, old_name)
            temp_path = os.path.join(folder_path, temp_name)

            if os.path.exists(old_path):
                os.rename(old_path, temp_path)

        # Then, rename temporary files to the final names
        for j in range(30):
            temp_name = f"temp_{str(j).zfill(2)}.png"
            new_name = f"panoramic_{str(29-j).zfill(2)}.png"
            temp_path = os.path.join(folder_path, temp_name)
            new_path = os.path.join(folder_path, new_name)

            if os.path.exists(temp_path):
                os.rename(temp_path, new_path)

        print(f"Finished processing folder {folder_name}")

    # Rezip the folders with the new name
    new_zip_name_parts = zip_name[:-4].split("_")  # Split the original zip name
    new_zip_name_parts[1], new_zip_name_parts[2] = (
        new_zip_name_parts[2],
        new_zip_name_parts[1],
    )  # Swap 'start' and 'end'
    new_zip_name = "_".join(new_zip_name_parts) + ".zip"

    with zipfile.ZipFile(os.path.join(base_path, new_zip_name), "w") as zipf:
        for i in range(300):
            folder_name = f"path{str(i).zfill(2)}"
            folder_path = os.path.join(base_path, folder_name)
            for root, dirs, files in os.walk(folder_path):
                for file in files:
                    zipf.write(
                        os.path.join(root, file),
                        os.path.relpath(os.path.join(root, file), base_path),
                    )

    # Optional: Delete the path folders after re-zipping
    for i in range(300):
        shutil.rmtree(os.path.join(base_path, f"path{str(i).zfill(2)}"))


# Usage
preprocess_image_data("./DemoExperiment/process", "demo_exterior_room_a.zip")

Finished processing folder path00
Finished processing folder path01
Finished processing folder path02
Finished processing folder path03
Finished processing folder path04
Finished processing folder path05
Finished processing folder path06
Finished processing folder path07
Finished processing folder path08
Finished processing folder path09
Finished processing folder path10
Finished processing folder path11
Finished processing folder path12
Finished processing folder path13
Finished processing folder path14
Finished processing folder path15
Finished processing folder path16
Finished processing folder path17
Finished processing folder path18
Finished processing folder path19
Finished processing folder path20
Finished processing folder path21
Finished processing folder path22
Finished processing folder path23
Finished processing folder path24
Finished processing folder path25
Finished processing folder path26
Finished processing folder path27
Finished processing folder path28
Finished proce

# Network Test

In [18]:
import os
import sys
import time
import re
import numpy as np
import random
from tqdm import tqdm
from tensorboardX import SummaryWriter
import matplotlib.pyplot as plt
import math
import zipfile
import sklearn
import shutil
from glob import glob
from PIL import Image

plt.switch_backend("agg")
%matplotlib inline
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
from torchvision import datasets, models, transforms
import torchvision.utils as vutils
import torch.nn.functional as F


In [19]:
import json
from helpers.augmentation import (
    BrightnessJitter,
    RandomHorizontalFlip,
    Scale,
    RandomCropWithProb,
)
from helpers.dataset import BldgDataset
from helpers.utils import (
    AverageMeter,
    save_checkpoint,
    Logger,
    neq_load_customized,
    MultiStepLR_Restart_Multiplier,
    calc_topk_accuracy,
)
from backbone.select_backbone import select_resnet
from backbone.memdpc import MemDPC_BD
from helpers.training_utils import get_data, set_path, train_one_epoch, validate

args = json.load(open("./config/demo_config.json"))


In [20]:
def main(args):
    torch.manual_seed(args["seed"])
    np.random.seed(args["seed"])
    random.seed(args["seed"])

    os.environ["CUDA_VISIBLE_DEVICES"] = str(args["gpu"])
    device = torch.device("cuda")
    num_gpu = len(str(args["gpu"]).split(","))
    print(f"num_gpu:{num_gpu}")
    # args["batch_size"] = num_gpu * args["batch_size"]

    ### model ###
    # need to test:
    # num_seq to 10?  seq_len to 3
    if args["model"] == "memdpc":
        model = MemDPC_BD(
            sample_size=args["img_dim"],
            num_seq=args["num_seq"],
            seq_len=args["seq_len"],
            network=args["net"],
            pred_step=args["pred_step"],
            mem_size=args["mem_size"],
            drop_out=args["drop_out"],
        )
    else:
        raise NotImplementedError("wrong model!")

    model.to(device)
    model = nn.DataParallel(model)
    model_without_dp = model.module

    ### optimizer ###
    params = model.parameters()
    optimizer = optim.Adam(params, lr=args["lr"], weight_decay=args["wd"])
    criterion = nn.CrossEntropyLoss()

    ### data ###
    train_transform = transforms.Compose(
        [
            RandomCropWithProb(size=[25, 50], p=args["p"], consistent=True),
            Scale(size=(30, 60)),
            RandomHorizontalFlip(consistent=True, p=args["p"]),
            BrightnessJitter(brightness=[0.5, 3], consistent=True, p=args["p"]),
            # RandomHorizontalShift(max_shift=60, p=args["p"]),
        ]
    )

    val_transform = None

    train_loader = get_data(train_transform, args=args, mode="train")
    val_loader = get_data(val_transform, args=args, mode="val")

    lr_milestones_eps = [50, 100]  # can be smaller

    lr_milestones = [len(train_loader) * m for m in lr_milestones_eps]
    print(
        "=> Use lr_scheduler: %s eps == %s iters"
        % (str(lr_milestones_eps), str(lr_milestones))
    )
    lr_lambda = lambda ep: MultiStepLR_Restart_Multiplier(
        ep, gamma=0.1, step=lr_milestones, repeat=1
    )
    lr_scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)

    best_acc = 0
    args["iteration"] = 1

    ### restart training ###
    if args["resume"]:
        if os.path.isfile(args["resume"]):
            print("=> loading resumed checkpoint '{}'".format(args["resume"]))
            checkpoint = torch.load(args["resume"], map_location=torch.device("cpu"))
            args["start_epoch"] = checkpoint["epoch"]
            args["iteration"] = checkpoint["iteration"]
            best_acc = checkpoint["best_acc"]
            model_without_dp.load_state_dict(checkpoint["state_dict"])
            try:
                optimizer.load_state_dict(checkpoint["optimizer"])
            except:
                print("[WARNING] Not loading optimizer states")
            print(
                "=> loaded resumed checkpoint '{}' (epoch {})".format(
                    args["resume"], checkpoint["epoch"]
                )
            )
        else:
            print("[Warning] no checkpoint found at '{}'".format(args["resume"]))
            sys.exit(0)

    # logging tools
    args["img_path"], args["model_path"] = set_path(args)
    args["logger"] = Logger(path=args["img_path"])
    args["logger"].log(
        "args=\n\t\t"
        + "\n\t\t".join(["%s:%s" % (str(k), str(v)) for k, v in args.items()])
    )

    args["writer_val"] = SummaryWriter(logdir=os.path.join(args["img_path"], "val"))
    args["writer_train"] = SummaryWriter(logdir=os.path.join(args["img_path"], "train"))

    torch.backends.cudnn.benchmark = True

    ### main loop ###
    for epoch in range(args["start_epoch"], args["epochs"]):
        np.random.seed(epoch)
        random.seed(epoch)

        train_loss, train_acc = train_one_epoch(
            train_loader, model, criterion, optimizer, lr_scheduler, device, epoch, args
        )
        val_loss, val_acc = validate(val_loader, model, criterion, device, epoch, args)

        # save check_point
        is_best = val_acc > best_acc
        best_acc = max(val_acc, best_acc)
        save_dict = {
            "epoch": epoch,
            "state_dict": model_without_dp.state_dict(),
            "best_acc": best_acc,
            "optimizer": optimizer.state_dict(),
            "iteration": args["iteration"],
        }
        save_checkpoint(
            save_dict,
            is_best,
            filename=os.path.join(args["model_path"], "epoch%s.pth.tar" % str(epoch)),
            keep_all=False,
        )
        args["best_val_acc"] = best_acc

    print(
        "Training from ep %d to ep %d finished" % (args["start_epoch"], args["epochs"])
    )
    # sys.exit(0)

In [21]:
def change_args(
    args,
    mem_size=128,
    epochs=60,
    batch_size=8,
    p=0.65,
    lr=5e-4,
    wd=1e-4,
    drop_out=0.3,
    data_path="./DemoExperiment/1117/demo_test.zip",
):  # try batch size
    args["mem_size"] = mem_size
    args["epochs"] = epochs
    # args['workers'] = 12
    args["batch_size"] = batch_size
    args["p"] = p
    args["lr"] = lr
    args["wd"] = wd
    args["data_path"] = data_path
    args["workers"] = 2
    args["drop_out"] = drop_out
    return args

In [22]:
args = change_args(args)
main(args)


num_gpu:2
Using MemDPC-BiDirectional model with resnet18_simplified and mem_size 128
final feature map has size 1x2
MEM Bank has size 128x128
Loading BldgDataset dataset for train


Loading demo: 100%|██████████| 300/300 [00:01<00:00, 225.74path/s]


"train" dataset size: 420
Loading BldgDataset dataset for val


Loading demo: 100%|██████████| 300/300 [00:03<00:00, 92.10path/s]


"val" dataset size: 90
=> Use lr_scheduler: [50, 100] eps == [2600, 5200] iters
Epoch: [0][0/52]	Loss 11.698868	Acc: 0.0208	T-data:5.95 T-batch:9.04	
Epoch: [0][5/52]	Loss 19.051331	Acc: 0.0208	T-data:0.00 T-batch:0.06	
Epoch: [0][10/52]	Loss 9.588809	Acc: 0.0417	T-data:0.00 T-batch:0.06	
Epoch: [0][15/52]	Loss 9.796906	Acc: 0.0417	T-data:0.00 T-batch:0.06	
Epoch: [0][20/52]	Loss 7.234867	Acc: 0.0208	T-data:0.00 T-batch:0.06	
Epoch: [0][25/52]	Loss 6.851819	Acc: 0.0417	T-data:0.00 T-batch:0.06	
Epoch: [0][30/52]	Loss 6.170689	Acc: 0.0417	T-data:0.00 T-batch:0.06	
Epoch: [0][35/52]	Loss 5.975263	Acc: 0.0833	T-data:0.00 T-batch:0.06	
Epoch: [0][40/52]	Loss 5.628435	Acc: 0.1042	T-data:0.00 T-batch:0.06	
Epoch: [0][45/52]	Loss 5.605404	Acc: 0.0833	T-data:0.00 T-batch:0.06	
Epoch: [0][50/52]	Loss 5.277074	Acc: 0.1667	T-data:0.00 T-batch:0.07	
Epoch: [0]	T-epoch:12.71	
Validation:
Epoch: [0/60]	Loss 5.291866	Acc: 0.0833	
Epoch: [1][0/52]	Loss 5.194293	Acc: 0.1042	T-data:4.14 T-batch:4.16	
Ep

: 

In [14]:
from helpers.dataset import BldgDataset
from helpers.augmentation import RandomHorizontalFlip, BrightnessJitter
import cv2
from torchvision import datasets, models, transforms
import numbers
import random
import cv2
import numpy as np
import collections

In [4]:
class RandomCropWithProb:
    def __init__(self, size, p=0.8, consistent=True):
        if isinstance(size, numbers.Number):
            self.size = (int(size), int(size))
        else:
            self.size = (size[0], size[1])
        self.consistent = consistent
        self.threshold = p

    def __call__(self, imgmap):
        cropped_images = []
        for img in imgmap:
            h, w = img.shape[-2:]  # Assuming img shape is (C, H, W)
            th, tw = self.size

            if w == tw and h == th:
                cropped_images.append(img)
                continue

            if self.consistent:
                if random.random() < self.threshold:
                    x1 = random.randint(0, w - tw)
                    y1 = random.randint(0, h - th)
                else:
                    x1 = int(round((w - tw) / 2.0))
                    y1 = int(round((h - th) / 2.0))
                cropped_img = img[:, y1 : y1 + th, x1 : x1 + tw]
            else:
                if random.random() < self.threshold:
                    x1 = random.randint(0, w - tw)
                    y1 = random.randint(0, h - th)
                else:
                    x1 = int(round((w - tw) / 2.0))
                    y1 = int(round((h - th) / 2.0))
                cropped_img = img[:, y1 : y1 + th, x1 : x1 + tw]

            cropped_images.append(cropped_img)

        return np.stack(cropped_images, axis=0)


class Scale:
    def __init__(self, size, interpolation=cv2.INTER_NEAREST):
        assert isinstance(size, int) or (len(size) == 2)
        self.size = size
        self.interpolation = interpolation

    def __call__(self, imgmap):
        resized_images = []
        for img in imgmap:
            h, w = img.shape[-2:]  # Assuming img shape is (C, H, W)
            if isinstance(self.size, int):
                if (w <= h and w == self.size) or (h <= w and h == self.size):
                    resized_images.append(img)
                    continue
                if w < h:
                    ow = self.size
                    oh = int(self.size * h / w)
                else:
                    oh = self.size
                    ow = int(self.size * w / h)
                resized_img = cv2.resize(
                    img[0], (ow, oh), interpolation=self.interpolation
                )
                resized_img = np.expand_dims(
                    resized_img, axis=0
                )  # Add channel dimension back
            else:
                resized_img = cv2.resize(
                    img[0],
                    (self.size[1], self.size[0]),
                    interpolation=self.interpolation,
                )
                resized_img = np.expand_dims(
                    resized_img, axis=0
                )  # Add channel dimension back

            resized_images.append(resized_img)

        return np.stack(resized_images, axis=0)


In [9]:
import json

args = json.load(open("./config/demo_config.json"))


In [ ]:
train_transform = transforms.Compose(
    [
        RandomCropWithProb(size=[25, 50], p=args["p"], consistent=True),
        Scale(size=(30, 60)),
        RandomHorizontalFlip(consistent=True, p=args["p"]),
        BrightnessJitter(brightness=[0.5, 3], consistent=True, p=args["p"]),
        # RandomHorizontalShift(max_shift=60, p=args["p"]),
    ]
)
test_data = BldgDataset(
    data_path="./DemoExperiment/1117/demo_test.zip", transform=train_transform
)


ValueError: all input arrays must have the same shape